# Notebook for DSW Telkomsel 2023 - Party Parrot

In [2]:
import os
import re
import duckdb
import numpy as np
import pandas as pd

from fuzzywuzzy import fuzz
from tqdm import tqdm

%load_ext autoreload
%autoreload 2

# Read Data

In [3]:
df_name = pd.read_excel('datasets/raw/product_name.xlsx')
catalog = pd.read_excel('datasets/raw/product_catalog.xlsx')

df_name.rename(columns={'Product Name': 'product_name'}, inplace=True)
catalog.rename(columns={
    'Product SKU': 'product_sku',
    'Brand': 'brand',
    'Type': 'type',
    'Formula': 'formula'
}, inplace=True)

df_name = df_name.dropna()
df_name

,product_name
0,Pupuk Urea N 46%
1,Pupuk Amonium Sulfat ZA
2,Pupuk Super Fosfat SP-36
3,Pupuk NPK Phonska
4,Pupuk NPK Formula Khusus
...,...
43997,Extra one 680 EC @ 250 ml
43998,Extra One 680 SC @ 500 ml
43999,JARING ARWANA @ 100 METER
44000,Terong Puma F1 @ 5 gram


In [4]:
catalog

,product_sku,brand,type,formula
0,Urea Petro,PIHC,Urea,NaN
1,Urea PIM,PIHC,Urea,NaN
2,Urea Nitrea,PIHC,Urea,NaN
3,Urea Daun Buah,PIHC,Urea,NaN
4,Urea Pusri,PIHC,Urea,NaN
...,...,...,...,...
182,Mestical,LaoYing,Nitrogen,NaN
183,Mesti-GAS,LaoYing,ZA,NaN
184,Mestac,LaoYing,Nitrogen,NaN
185,MestiKisrit,LaoYing,Mg,NaN


# Preprocessing

In [24]:
def is_containing_non_alphanumeric(string):
    return not bool(re.search(r'[^a-zA-Z0-9 ]', string))

df_name['is_only_alphanum'] = df_name['product_name'].apply(is_containing_non_alphanumeric)
catalog['is_only_alphanum'] = catalog['product_sku'].apply(is_containing_non_alphanumeric)

In [31]:
def clean(s):
    s = s.lower()

    # give spaces before and after the special characters, that are not number or decimal number
    # e.g. 'abc+c' to 'abc + c', but '12.9' not to '12 . 9'
    s = re.sub(r'([^\d\.,-]+)', r' \1 ', s)

    # separate value and units like "100ml" to "100 ml" or "50g" to "50 g"
    # that are not wrapped in <...> bracket
    s = re.sub(r'(\d+)([a-zA-Z]+)', r'\1 \2', s)

    # make chemical formula separator uniform to 'x', i.e. '4.5 - 3.6 - 2.1' to '4.5x3.6x2.1'
    s = re.sub(r'(\d+\.?,?\d*)[×x\+,\. |-]+(\d+\.?,?\d*)[×x\+,\. |-]+(\d+\.?,?\d*)', r'\1x\2x\3', s)

    # remove extra spaces
    s = re.sub(r'\s+', ' ', s)
    
    return s

catalog['clean_sku'] = catalog.product_sku.apply(clean)
catalog['brand'] = catalog.brand.str.lower().str.replace(' ', '')

unique_brand = sorted(catalog.brand.unique())
df_name['clean_name'] = df_name.product_name.apply(clean)

In [45]:
brand_sku_tokens = {}

for brand in unique_brand:
    brand_sku_tokens[brand] = set()

    for sku in catalog[catalog.brand == brand].clean_sku:
        brand_sku_tokens[brand] |= set(sku.split(' '))

for brand in brand_sku_tokens:
    # keep alphabetics with length >= 4 only
    brand_sku_tokens[brand] = set(filter(lambda x: len(x) >= 4 and x.isalpha(), brand_sku_tokens[brand]))

brand_sku_tokens['dgw/hextar'] |= {'hx', 'dgw'}
# brand_sku_tokens

In [7]:
df_name[df_name.is_only_alphanum == False].loc[5200:5400]

,product_name,is_only_alphanum,clean_name
5203,npk kujang 30.6.8 25kg,False,npk kujang 30.6x8x25 kg
5211,Pakan Ayam 311/511/512,False,pakan ayam 311 / 511 / 512
5213,Super-K 500 grm,False,super - k 500 grm
5218,Lanet 25 wp. 15 grm,False,lanet 25 wp . 15 grm
5220,Lannete 40 SP. 100 grm,False,lannete 40 sp . 100 grm
5225,X-TRAIL 100ml,False,x - trail 100 ml
5243,selang putih 0.20 x 15 x 20,False,selang putih 0.20x15x20
5244,selang putih 0.20x20x50,False,selang putih 0.20x20x50
5245,"0,20×15×50 putih",False,"0,20x15x50 putih"
5249,sp26 -25kg,False,sp 26 -25 kg


In [8]:
catalog[catalog.is_only_alphanum == False].head(20)

,product_sku,brand,type,formula,is_only_alphanum,clean_sku
9,Petro-CAS,PIHC,Mikro,NaN,False,petro - cas
10,SP-36 Petro,PIHC,Fosfat,NaN,False,sp -36 petro
13,SP-26 Petro,PIHC,Fosfat,NaN,False,sp -26 petro
15,Phonska Plus 15-15-15+9S+0.2Zn,PIHC,Majemuk,15-15-15,False,phonska plus 15x15x15 + 9 s+ 0.2 zn
16,NPK Kebomas 12-12-17+2MgO+0.1Zn+0.2B+0.2Fe,PIHC,Majemuk,12-12-17,False,npk kebomas 12x12x17 + 2 mgo+ 0.1 zn+ 0.2 b+ ...
17,NPK Kebomas 12-6-22+3Mg,PIHC,Majemuk,12-6-22,False,npk kebomas 12x6x22 + 3 mg
18,NPK Kebomas 15-15-15,PIHC,Majemuk,15-15-15,False,npk kebomas 15x15x15
19,PETROFERT 16-16-8+13S,PIHC,Majemuk,16-16-8,False,petrofert 16x16x8 + 13 s
20,Petro Niphos 20-20+13S,PIHC,Majemuk,20-20-0,False,petro niphos 20x20x13 s
21,FERTIGRES 16-20+13S,PIHC,Majemuk,16-20-0,False,fertigres 16x20x13 s


In [9]:
df_name.to_csv('datasets/processed/product_name.tsv', sep='\t', index=False)
catalog.to_csv('datasets/processed/product_catalog.tsv', sep='\t', index=False)

# Levenshtein Search

In [10]:
joined = duckdb.query(
    '''
    WITH joined AS (
        SELECT
            dn.clean_name,
            c.product_sku,
            c.clean_sku
        FROM
            df_name AS dn
        CROSS JOIN
            catalog AS c
    )

    SELECT
        clean_name,
        product_sku,
        clean_sku,
        levenshtein(clean_name, clean_sku) AS lev_distance
    FROM joined
    '''
).to_df()

joined

,clean_name,product_sku,clean_sku,lev_distance
0,pupuk urea n 46 %,Urea Petro,urea petro,12
1,pupuk amonium sulfat za,Urea Petro,urea petro,19
2,pupuk super fosfat sp -36,Urea Petro,urea petro,19
3,pupuk npk phonska,Urea Petro,urea petro,13
4,pupuk npk formula khusus,Urea Petro,urea petro,20
...,...,...,...,...
8228182,extra one 680 ec @ 250 ml,Borate Evermax,borate evermax,20
8228183,extra one 680 sc @ 500 ml,Borate Evermax,borate evermax,21
8228184,jaring arwana @ 100 meter,Borate Evermax,borate evermax,21
8228185,terong puma f 1 @ 5 gram,Borate Evermax,borate evermax,20


In [11]:
result_lev = duckdb.query(
    """
    SELECT
        *
    FROM
        joined
    WHERE
        lev_distance < 2
    QUALIFY
        ROW_NUMBER() OVER (PARTITION BY clean_name ORDER BY lev_distance, clean_sku) = 1
    """
).to_df()

result_lev

,clean_name,product_sku,clean_sku,lev_distance
0,urea netrea,Urea Nitrea,urea nitrea,1
1,za pertro,ZA Petro,za petro,1
2,usp,KSP,ksp,1
3,mesti - za,Mesti-ZA,mesti - za,0
4,npk pusri 15x15x15,NPK Pusri 15-15-15,npk pusri 15x15x15,0
...,...,...,...,...
123,sp -36 petro,SP-36 Petro,sp -36 petro,0
124,gsp,KSP,ksp,1
125,phosgrow,Phosgro,phosgro,1
126,mutiara grower 18x09x20 +te,Mutiara GROWER 15-09-20+TE,mutiara grower 15x09x20 +te,1


In [12]:
final_res = df_name.copy()

final_res = duckdb.query(
    '''
    SELECT
        f.product_name,
        r.product_sku,
        r.clean_name,
        r.clean_sku,
        f.is_only_alphanum AS is_name_only_alphanum,
        r.lev_distance,
    FROM
        final_res AS f
    LEFT JOIN
        result_lev AS r
    ON
        f.clean_name = r.clean_name
    '''
).to_df()

final_res.iloc[-20:]

,product_name,product_sku,clean_name,clean_sku,is_name_only_alphanum,lev_distance
43981,Mutiara Grower 15-09-20+TE,Mutiara GROWER 15-09-20+TE,mutiara grower 15x09x20 +te,mutiara grower 15x09x20 +te,False,0.0
43982,YARAMILA UNIK 16.16.16,YaraMila UNIK 16-16-16,yaramila unik 16x16x16,yaramila unik 16x16x16,False,0.0
43983,Pelangi 16-16-16,Pelangi 16-16-16,pelangi 16x16x16,pelangi 16x16x16,False,0.0
43984,lsp,KSP,lsp,ksp,True,1.0
43985,mutiara 16 .16.16,Mutiara 16-16-16,mutiara 16x16x16,mutiara 16x16x16,False,0.0
43986,Nitrophonska 15-15-15,NITROPHOSKA 15-15-15,nitrophonska 15x15x15,nitrophoska 15x15x15,False,1.0
43987,NPK pim 15 15 15,NPK PIM 15-15-15,npk pim 15x15x15,npk pim 15x15x15,True,0.0
43988,verea,Vrea,verea,vrea,True,1.0
43989,Bsp,KSP,bsp,ksp,True,1.0
43990,fetiphos,Fertiphos,fetiphos,fertiphos,True,1.0


In [13]:
final_res.to_csv('datasets/processed/final_result.tsv', sep='\t', index=False)
final_res

,product_name,product_sku,clean_name,clean_sku,is_name_only_alphanum,lev_distance
0,Pupuk Urea N 46%,None,None,None,False,NaN
1,Pupuk Amonium Sulfat ZA,None,None,None,True,NaN
2,Pupuk Super Fosfat SP-36,None,None,None,False,NaN
3,Pupuk NPK Phonska,None,None,None,True,NaN
4,Pupuk NPK Formula Khusus,None,None,None,True,NaN
...,...,...,...,...,...,...
43996,NPK KEBOMAS 15 15 15,NPK Kebomas 15-15-15,npk kebomas 15x15x15,npk kebomas 15x15x15,True,0.0
43997,MerokeMKP,MerokeMOP,merokemkp,merokemop,True,1.0
43998,Petro Bio Fertil,Petro BioFertil,petro bio fertil,petro biofertil,True,1.0
43999,SP-26 Petro,SP-26 Petro,sp -26 petro,sp -26 petro,False,0.0


# Post-analysis

In [78]:
final_res = pd.read_csv('datasets/processed/final_result_2.tsv', sep='\t')
final_res

,product_name,product_sku,clean_name,clean_sku,is_name_only_alphanum,possible_brand,lev_distance,fuzzy_ratio
0,Pupuk Urea N 46%,Urea Daun Buah,pupuk urea n 46 %,urea daun buah,False,pihc,11,51.0
1,Pupuk Amonium Sulfat ZA,Ammonium Sulfate,pupuk amonium sulfat za,ammonium sulfate,True,NaN,10,74.0
2,Pupuk Super Fosfat SP-36,Triple Super Phospate (TSP),pupuk super fosfat sp -36,triple super phospate (tsp),False,mahkota,15,62.0
3,Pupuk NPK Formula Khusus,Pak Tani Fertila 18-6-14+5S,pupuk npk formula khusus,pak tani fertila 18x6x14 + 5 s,True,NaN,17,45.0
4,Pupuk Organik Granul,Urea Nitrea,pupuk organik granul,urea nitrea,True,NaN,14,51.0
...,...,...,...,...,...,...,...,...
43996,grentonik 1ltr,MerokeCPN NK 15-15+TE,grentonik 1 ltr,merokecpn nk 15-15 +te,True,NaN,10,54.0
43997,methachlor 400ml,Mestac,methachlor 400 ml,mestac,True,NaN,12,52.0
43998,Urea Pril 50Kg PKT,Urea Pusri,urea pril 50 kg pkt,urea pusri,True,pihc,12,61.0
43999,Mutiara 20-10-10,Mutiara SPRINTER 20-10-10,mutiara 20x10x10,mutiara sprinter 20x10x10,False,mutiara,4,79.0


In [86]:
cond = (
      (final_res.possible_brand.notnull() & (final_res.fuzzy_ratio >= 54))
    | (final_res.possible_brand.isnull()  & (final_res.fuzzy_ratio >= 70))
)

# final_res[cond]
final_res[final_res.possible_brand.isnull() & (final_res.fuzzy_ratio.between(60, 69))].iloc[list(range(10)) + list(range(-10, 0))]

,product_name,product_sku,clean_name,clean_sku,is_name_only_alphanum,possible_brand,lev_distance,fuzzy_ratio
13,puradan,Urea Daun Buah,puradan,urea daun buah,True,NaN,5,64.0
19,fastak,Mestac,fastak,mestac,True,NaN,3,62.0
26,REGENT,Phosgreen,regent,phosgreen,True,NaN,5,63.0
28,manuver,YaraVera,manuver,yaravera,True,NaN,4,63.0
31,rengt,Urecote,rengt,urecote,True,NaN,4,62.0
33,sidacin,Mestikali,sidacin,mestikali,True,NaN,6,60.0
34,klocyper,ZK Petro,klocyper,zk petro,True,NaN,6,60.0
37,fenval,Fertikali,fenval,fertikali,True,NaN,4,63.0
40,Dipho,Fertiphos,dipho,fertiphos,True,NaN,4,67.0
42,anteb,Urea Nitrea,anteb,urea nitrea,True,NaN,4,60.0
